<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-09 11:15:48
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  9.00 K
Current:  1.37 C
-------------------
Today PnL: 45.15 K (0.33%)
Current PnL: -12.77 L (-8.9%)
CY Booked + Current PnL: -5.81 L (-4.05%)
-------------------
Total profit:  5.04 L
Total loss:  -17.81 L
-------------------
Total Booked + Current PnL: 21.17 L (18.17%)
Total Booked PnL: 33.94 L (29.12%)
Curr Year Booked PnL: 6.96 L (5.07%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 71.28 L (51.9%)
Deployed:  1.17 C
Current:  1.37 C
CAGR/XIRR %: 11.07%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.00,-217.75,H-SC,98.96,84981.0,-10335.0,10376.0,3.44,-10.84,12.21,0.04,157.0,-1.00,0.62,67.62,OX40N,NTT,MISC
4,APOLLOHOSP,6904.43,8285.00,-17.29,H-LC,90.62,112718.0,9152.0,11554.0,-0.67,8.84,10.25,20.00,32.0,0.79,0.82,24.15,X40N,BTT,HEALTHCARE
39,HINDALCO,651.95,761.55,-21.55,H-LC,75.00,108512.0,4200.0,13336.0,-1.08,4.03,12.29,16.81,27.0,0.31,0.79,20.72,X5K,ATH,METALS
71,SBIN,760.30,863.00,-11.51,M-LC,30.21,215114.0,13634.0,13574.0,-0.11,6.77,6.31,13.51,77.0,1.00,1.57,20.22,XY25,NTT,BANKS
87,UJJIVANSFB,52.77,53.00,45.86,M-SC,19.79,126333.0,-16146.0,16764.0,-0.72,-11.33,13.27,0.44,247.0,-0.96,0.92,49.11,OX40N,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,116.91,190.00,-32.52,M-MC,7.29,178731.0,-1077.0,113494.0,-0.64,-0.60,63.50,62.52,173.0,-0.01,1.30,33.28,XR,NTT,BANKS
38,HEROMOTOCO,4311.81,6039.03,-6.19,H-MC,36.46,151200.0,287.0,60162.0,0.46,0.19,39.79,40.06,103.0,0.00,1.10,23.67,AR,ATH,AUTO
60,NATIONALUM,189.63,247.44,-50.15,H-MC,48.96,100090.0,-414.0,31048.0,-0.56,-0.41,31.02,30.49,97.0,-0.01,0.73,31.86,MH,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,MEDANTA,1064.65,1486.00,16.28,X-SC,87.50,247812.0,42335.0,38981.0,5.41,20.60,15.73,39.58,65.0,1.09,1.81,32.80,XY24,NTT,HEALTHCARE
54,KPIGREEN,497.21,731.79,29.10,H-SC,59.38,132212.0,6915.0,52197.0,3.44,5.52,39.48,47.18,143.0,0.13,0.96,65.41,MH,ATH,POWER
92,VIPIND,488.80,489.00,-217.75,H-SC,98.96,84981.0,-10335.0,10376.0,3.44,-10.84,12.21,0.04,157.0,-1.00,0.62,67.62,OX40N,NTT,MISC
94,WHIRLPOOL,1167.49,2270.00,-38.81,M-SC,51.04,201669.0,30048.0,132013.0,2.92,17.51,65.46,94.43,201.0,0.23,1.47,48.84,XR,NTT,DURABLES
55,LALPATHLAB,2804.19,3545.00,-29.39,H-SC,66.15,162415.0,8185.0,32564.0,2.53,5.31,20.05,26.42,127.0,0.25,1.18,28.08,X40N,NTT,HEALTHCARE


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,UNIONBANK,123.87,163.00,-6.01,M-LC,6.25,163535.0,22695.0,21799.0,-4.27,16.11,13.33,31.59,88.0,1.04,1.19,42.05,XY24,NTT,BANKS
0,5PAISA,564.75,565.00,148.80,H-SC,17.71,134535.0,-52397.0,52482.0,-2.35,-28.03,39.01,0.04,149.0,-1.00,0.98,28.70,OX40N,NTT,FINANCE
90,VALIANTORG,512.64,838.00,-357.02,H-SC,16.67,109382.0,-23904.0,108496.0,-1.82,-17.93,99.19,63.47,151.0,-0.22,0.80,83.35,XR,NTT,CHEMICALS
28,DIGITIDE,188.38,237.72,NaN,NaN,28.12,50153.0,8898.0,1906.0,-1.69,21.57,3.80,26.19,272.0,4.67,0.37,18.08,XY24,ATH,IT
68,SAIL,130.64,228.00,90.28,M-MC,34.38,228819.0,3857.0,163789.0,-1.61,1.71,71.58,74.53,189.0,0.02,1.67,32.85,XY24,BTT,STEEL


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.0,-217.75,H-SC,98.96,84981.0,-10335.0,10376.0,3.44,-10.84,12.21,0.04,157.0,-1.00,0.62,67.62,OX40N,NTT,MISC
87,UJJIVANSFB,52.77,53.0,45.86,M-SC,19.79,126333.0,-16146.0,16764.0,-0.72,-11.33,13.27,0.44,247.0,-0.96,0.92,49.11,OX40N,NTT,BANKS
79,SYMPHONY,1306.42,1306.0,-28.99,M-SC,24.48,151253.0,-19888.0,19829.0,0.05,-11.62,13.11,-0.03,191.0,-1.00,1.10,8.71,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,92.71,149048.0,-25511.0,25547.0,0.14,-14.61,17.14,0.02,133.0,-1.00,1.09,29.25,OX40N,NTT,PAINTS
81,TATAELXSI,7332.28,7332.0,-18.15,X-MC,8.33,85848.0,-16804.0,16800.0,-0.64,-16.37,19.57,-0.00,59.0,-1.00,0.63,30.50,OX40N,NTT,IT


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TCS,3794.03,4998.00,-18.70,X-LC,35.42,265044.0,-30890.0,124809.0,-0.24,-10.44,47.09,31.73,1.0,-0.25,1.93,6.09,X40,BTT,IT
48,INFY,1461.46,2275.00,-8.83,X-LC,72.92,292200.0,30599.0,115039.0,-0.38,11.70,39.37,55.67,2.0,0.27,2.13,18.52,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-13.47,X-LC,60.42,270984.0,637.0,33981.0,1.12,0.24,12.54,12.81,5.0,0.02,1.97,13.33,XY25,NTT,FMCG
17,BRITANNIA,4983.33,6446.05,17.40,X-LC,64.58,269008.0,39775.0,27520.0,0.15,17.35,10.23,29.35,10.0,1.45,1.96,27.82,XY25,ATH,FMCG
74,SIEMENS,4406.90,7969.85,-3.50,X-LC,21.88,81648.0,-28524.0,117598.0,-0.94,-25.89,144.03,80.85,11.0,-0.24,0.59,20.25,AR,ATH,ELECTRICAL


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3089.70,NaN,NaN,95.83,77242.0,42793.0,0.0,1.40,124.22,0.00,124.22,274.0,inf,0.56,18.12,AR,ATH,ELECTRICAL
28,DIGITIDE,188.38,237.72,NaN,NaN,28.12,50153.0,8898.0,1906.0,-1.69,21.57,3.80,26.19,272.0,4.67,0.37,18.08,XY24,ATH,IT
52,JPPOWER,18.73,26.20,-15.28,L-SC,100.00,170695.0,28853.0,27721.0,0.67,20.34,16.24,39.88,262.0,1.04,1.24,72.99,XY24,NTT,POWER
59,MEDANTA,1064.65,1486.00,16.28,X-SC,87.50,247812.0,42335.0,38981.0,5.41,20.60,15.73,39.58,65.0,1.09,1.81,32.80,XY24,NTT,HEALTHCARE


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,90.28,M-MC,34.38,228819.0,3857.0,163789.0,-1.61,1.71,71.58,74.53,189.0,0.02,1.67,32.85,XY24,BTT,STEEL
54,KPIGREEN,497.21,731.79,29.10,H-SC,59.38,132212.0,6915.0,52197.0,3.44,5.52,39.48,47.18,143.0,0.13,0.96,65.41,MH,ATH,POWER
20,CAMS,3643.00,5250.99,4.54,H-SC,26.04,117505.0,15501.0,29517.0,1.81,15.20,25.12,44.14,126.0,0.53,0.86,33.91,X40N,ATH,MISC
95,WIPRO,243.46,420.00,-2.86,M-LC,85.42,166036.0,15091.0,94358.0,-0.69,10.00,56.83,72.51,71.0,0.16,1.21,14.37,XR,NTT,IT
15,BLUESTARCO,1646.70,2337.55,-3.28,H-SC,96.88,181270.0,16600.0,52478.0,0.14,10.08,28.95,41.95,124.0,0.32,1.32,18.30,X40N,ATH,AC


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3089.70,NaN,NaN,95.83,77242.0,42793.0,0.0,1.40,124.22,0.00,124.22,274.0,inf,0.56,18.12,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-38.81,M-SC,51.04,201669.0,30048.0,132013.0,2.92,17.51,65.46,94.43,201.0,0.23,1.47,48.84,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-2.86,M-LC,85.42,166036.0,15091.0,94358.0,-0.69,10.00,56.83,72.51,71.0,0.16,1.21,14.37,XR,NTT,IT
54,KPIGREEN,497.21,731.79,29.10,H-SC,59.38,132212.0,6915.0,52197.0,3.44,5.52,39.48,47.18,143.0,0.13,0.96,65.41,MH,ATH,POWER
45,INDIAMART,2327.09,4911.36,-47.70,H-SC,73.96,136062.0,12726.0,124238.0,0.53,10.32,91.31,111.05,123.0,0.10,0.99,33.32,AR,ATH,MISC


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,86.12,NaN,NaN,57.29,18860.0,-30733.0,0.0,1.65,-61.97,0.00,-61.97,270.0,-inf,0.14,26.48,XY24,ATH,MISC
24,COFFEEDAY,59.14,80.00,-55.73,L-SC,62.50,64896.0,-48653.0,88706.0,0.75,-42.85,136.69,35.27,265.0,-0.55,0.47,58.83,XR,NTT,HOTELS
32,ENRIN,1377.95,3089.70,NaN,NaN,95.83,77242.0,42793.0,0.0,1.40,124.22,0.00,124.22,274.0,inf,0.56,18.12,AR,ATH,ELECTRICAL
92,VIPIND,488.80,489.00,-217.75,H-SC,98.96,84981.0,-10335.0,10376.0,3.44,-10.84,12.21,0.04,157.0,-1.00,0.62,67.62,OX40N,NTT,MISC
58,MASFIN,326.60,399.50,-17.80,H-SC,69.79,93420.0,-4560.0,26429.0,-0.06,-4.65,28.29,22.32,145.0,-0.17,0.68,35.49,XR,ATH,FINANCE


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-15.28,L-SC,100.00,170695.0,28853.0,27721.0,0.67,20.34,16.24,39.88,262.0,1.04,1.24,72.99,XY24,NTT,POWER
92,VIPIND,488.80,489.00,-217.75,H-SC,98.96,84981.0,-10335.0,10376.0,3.44,-10.84,12.21,0.04,157.0,-1.00,0.62,67.62,OX40N,NTT,MISC
3,ALKYLAMINE,2347.98,4546.37,7.60,X-SC,97.92,100177.0,-786.0,95318.0,-1.11,-0.78,95.15,93.63,67.0,-0.01,0.73,49.21,SR,ATH,CHEMICALS
15,BLUESTARCO,1646.70,2337.55,-3.28,H-SC,96.88,181270.0,16600.0,52478.0,0.14,10.08,28.95,41.95,124.0,0.32,1.32,18.30,X40N,ATH,AC
32,ENRIN,1377.95,3089.70,NaN,NaN,95.83,77242.0,42793.0,0.0,1.40,124.22,0.00,124.22,274.0,inf,0.56,18.12,AR,ATH,ELECTRICAL


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.98
1,25,41.98
2,50,70.62


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.52
LC    33.83
MC    21.54
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.01
X40      12.92
XY25     12.05
X40N     11.98
XR        9.99
OX40N     8.15
AR        8.14
MH        1.69
X5K       1.42
X200      1.40
SR        1.21
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.51
M-SC    15.86
H-LC    14.11
X-LC    12.88
H-MC     9.59
X-MC     7.57
M-LC     5.80
X-SC     4.53
M-MC     4.03
L-SC     2.62
L-LC     1.04
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.26,0.17,33.19
IT,12.07,-9.89,63.90
FINANCE,8.51,-11.39,51.81
BANKS,7.61,-9.06,53.55
MISC,5.90,-19.03,74.86
PAINTS,5.70,-11.60,33.60
ELECTRICAL,5.43,1.37,45.07
HEALTHCARE,4.74,1.04,27.87
AUTO,4.28,-15.09,65.69


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2635423.0,26
XR,1084444.0,13
AR,1030577.0,10
X40,679675.0,10
X40N,519111.0,11
XY25,464368.0,8
OX40N,311130.0,11
SR,178226.0,2
MH,83245.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1969777.0,20
M-SC,1765993.0,21
H-LC,603348.0,12
X-LC,540455.0,8
H-MC,517716.0,8
M-MC,417074.0,3
X-MC,412729.0,6
X-SC,296171.0,5
M-LC,265360.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      884912.0      6
M-SC       XY24      806856.0      7
H-SC       AR        527870.0      3
M-SC       XR        333672.0      4
M-MC       XY24      303580.0      2
H-MC       XY24      294371.0      3
X-LC       X40       275570.0      3
H-SC       XR        215621.0      3
           X40N      200772.0      4
H-LC       X40N      192076.0      4
M-SC       AR        187157.0      2
           OX40N     166617.0      5
M-LC       XY24      157428.0      2
X-MC       X40       156884.0      2
H-LC       X40       153274.0      3
L-SC       XR        150628.0      2
X-SC       XR        122564.0      1
M-SC       XY25      119556.0      1
X-LC       AR        117598.0      1
M-MC       XR        113494.0      1
X-MC       XY25      102994.0      1
X-LC       XY25       99867.0      3
X-SC       SR         95318.0      1
M-LC       XR         94358.0      1
H-SC       OX40N      88405.0      3
X-MC       XY24       83048.0      1
M-SC       SR         82908.0      1
H-MC       XY25       81575.0      1
H-LC       AR         79709.0      1
           X5K        73123.0      2
M-SC       X40        69227.0      1
H-LC       X200       68546.0      1
H-MC       AR         60162.0      1
L-SC       AR         58081.0      1
L-MC       XR         54107.0      1
X-MC       X40N       53003.0      1
H-SC       MH         52197.0      1
X-LC       X40N       47420.0      1
L-LC       XY25       46802.0      1
X-SC       OX40N      39308.0      2
           XY24       38981.0      1
H-LC       XY24       36620.0      1
H-MC       MH         31048.0      1
L-SC       XY24       27721.0      1
H-MC       X40N       25840.0      1
           X40        24720.0      1
X-MC       OX40N      16800.0      1
M-LC       XY25       13574.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
